<a href="https://colab.research.google.com/github/Zainabb110/CS584-Project/blob/main/AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive 
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [ ]:
train=pd.read_csv('gdrive/My Drive/Colab Notebooks/expenses.csv')

train.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 205 kB 30.1 MB/s 
     |████████████████████████████████| 2.0 MB 61.0 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split( train, test_size=0.2, random_state=42,shuffle=True)
train.shape, test.shape

((1070, 7), (268, 7))

In [ ]:
train[train.isna().any(axis=1)]

,age,sex,bmi,children,smoker,region,charges


In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
y = train.pop('charges')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)


In [ ]:
automl.fit(X_train, y_train, task="regression",metric='rmse',time_budget=900)

Streaming output truncated to the last 5000 lines.
[flaml.automl: 11-16 00:19:50] {3350} INFO -  at 600.0s,	estimator xgb_limitdepth's best error=4324.7612,	best estimator xgb_limitdepth's best error=4324.7612
INFO:flaml.automl: at 600.0s,	estimator xgb_limitdepth's best error=4324.7612,	best estimator xgb_limitdepth's best error=4324.7612
[flaml.automl: 11-16 00:19:50] {3164} INFO - iteration 2164, current learner xgb_limitdepth
INFO:flaml.automl:iteration 2164, current learner xgb_limitdepth
[flaml.automl: 11-16 00:19:50] {3350} INFO -  at 600.3s,	estimator xgb_limitdepth's best error=4324.7612,	best estimator xgb_limitdepth's best error=4324.7612
INFO:flaml.automl: at 600.3s,	estimator xgb_limitdepth's best error=4324.7612,	best estimator xgb_limitdepth's best error=4324.7612
[flaml.automl: 11-16 00:19:50] {3164} INFO - iteration 2165, current learner xgb_limitdepth
INFO:flaml.automl:iteration 2165, current learner xgb_limitdepth
[flaml.automl: 11-16 00:19:51] {3350} INFO -  at 600.

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best rmse on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgb_limitdepth
Best hyperparmeter config: {'n_estimators': 37, 'max_depth': 3, 'min_child_weight': 1.1335197557671517, 'learning_rate': 0.13699591308042144, 'subsample': 0.7332681066109121, 'colsample_bylevel': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.00480797303893628, 'reg_lambda': 6.333846258677824}
Best rmse on validation data: 4325
Training duration of best run: 0.04843 s


In [ ]:
y_val=test.pop('charges')
X_val= test

In [ ]:
from flaml.ml import sklearn_metric_loss_score
print('train rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_train), y_train))
print(' test rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_test), y_test))
print('  val rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_val), y_val))

train rmse = 3989.857119150838
 test rmse = 5276.557015196491
  val rmse = 4391.845855357303


In [ ]:
from sklearn.metrics import r2_score
print("R2: ", r2_score(y_val, automl.predict(X_val)))

R2:  0.8757586626835583
